In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 학습 모델 저장을 위한 라이브러리
import pickle

from sklearn.metrics import classification_report, accuracy_score

In [2]:
# 학습이 완료된 모델을 저장할 파일 이름
best_model_path = '머신러닝/channel_info_classification.dat'

# 교차검증 횟수
cv_count = 2

# 교차 검증
kfold = KFold(n_splits=cv_count, shuffle=True, random_state=1)

# 평가 결과를 담을 리스트
# 필요하다면 다른 것도 만들기
f1_score_list = []

# 학습 모델 이름
model_name_list = []

In [3]:
# 데이터 로드
df = pd.read_parquet('채널정보_전처리.parquet')

# 기준년월 datetime 변환
df['기준년월'] = pd.to_datetime(df['기준년월'], format='%Y%m')

# 12월 데이터만 필터
df = df[df['기준년월'].dt.month == 12].reset_index(drop=True)

# train/test 분리
train_df = df[~df['ID'].str.startswith('TEST_')].reset_index(drop=True)
test_df  = df[df['ID'].str.startswith('TEST_')].reset_index(drop=True)

In [4]:
# ID 저장
test_ids = test_df['ID'].copy()

# train 데이터로 OS구분코드 인코더 학습 후 변환
os_le = LabelEncoder().fit(train_df['OS구분코드'])
train_df['OS구분코드_enc'] = os_le.transform(train_df['OS구분코드'])
test_df ['OS구분코드_enc'] = os_le.transform(test_df ['OS구분코드'])

# 불필요한 컬럼 제거하고 결측값을 -1로 채우기
drop_cols = ['ID', '기준년월', 'OS구분코드']
train_df = train_df.drop(columns=drop_cols).fillna(-1)
test_df  = test_df.drop(columns=drop_cols).fillna(-1)

# train 데이터의 Segment를 문자열로 통일하고 인코딩
train_df['Segment'] = train_df['Segment'].astype(str)
seg_le = LabelEncoder().fit(train_df['Segment'])
train_y = seg_le.transform(train_df['Segment'])

# test 데이터에서 Segment 컬럼 제거
test_df = test_df.drop(columns=['Segment'], errors='ignore')

# 모델 입력에 사용할 피처 목록 정의
feature_cols = train_df.columns.drop('Segment')

# train과 test의 입력 데이터 분리
train_X = train_df[feature_cols]
test_X  = test_df[feature_cols]

# 입력 데이터 표준화
scaler = StandardScaler().fit(train_X)
train_X = scaler.transform(train_X)
test_X  = scaler.transform(test_X)

### XGBoost

In [5]:
xgboost_basic_model = XGBClassifier(verbose=-1, silent=True, tree_method='gpu_hist')

# 모델 학습 수행
xgboost_basic_model.fit(train_X, train_y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [6]:
# 학습 데이터에 대한 예측 및 성능 확인
y_train_pred = xgboost_basic_model.predict(train_X)
print("Train Accuracy:", accuracy_score(train_y, y_train_pred))
print(classification_report(train_y, y_train_pred, target_names=seg_le.classes_))

# 테스트 데이터에 대한 예측
test_preds_num = xgboost_basic_model.predict(test_X)
test_preds = seg_le.inverse_transform(test_preds_num)

# 결과를 ID와 함께 데이터프레임으로 정리
submission = pd.DataFrame({
    'ID': test_ids,
    'Segment': test_preds
})

Train Accuracy: 0.810945
              precision    recall  f1-score   support

           A       1.00      0.21      0.35       162
           B       1.00      0.50      0.67        24
           C       0.70      0.08      0.14     21265
           D       0.64      0.07      0.12     58207
           E       0.81      1.00      0.90    320342

    accuracy                           0.81    400000
   macro avg       0.83      0.37      0.43    400000
weighted avg       0.78      0.81      0.74    400000



In [7]:
# feature importance 추출 및 저장
importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': xgboost_basic_model.feature_importances_
}).sort_values('importance', ascending=False)
importance_df.to_csv('채널정보_XG_feature_importances.csv', index=False, encoding='utf-8-sig')

In [8]:
# 절대값 importance 배열
imp = xgboost_basic_model.feature_importances_

# 퍼센트로 변환
rel_imp = imp / imp.sum()

# DataFrame 생성
df_imp = pd.DataFrame({
    'feature': feature_cols,
    'absolute': imp,
    'relative': rel_imp
}).sort_values('relative', ascending=False)

# 확인
print(df_imp.head(10))

               feature  absolute  relative
53          OS구분코드_enc  0.200896  0.200896
40       불만제기후경과월_R12M  0.186036  0.186036
6          방문월수_PC_R6M  0.096466  0.096466
0         인입일수_ARS_R6M  0.045514  0.045514
43      당사멤버쉽_방문월수_R6M  0.037083  0.037083
10        방문후경과월_앱_R6M  0.031425  0.031425
7        방문후경과월_PC_R6M  0.028422  0.028422
44       홈페이지_금융건수_R6M  0.023651  0.023651
49  이용메뉴건수_ARS_R6M_num  0.023561  0.023561
39       IB문의건수_CS_R6M  0.020801  0.020801


In [9]:
# OS구분코드 인코딩 값으로 변경
df['OS구분코드'] = os_le.transform(df['OS구분코드'])

# 상대 중요도 ≥0.01 피처 리스트 생성
important_features = [
    'OS구분코드' if f == 'OS구분코드_enc' else f
    for f in df_imp.loc[df_imp['relative'] >= 0.01, 'feature']
]

# 저장할 컬럼명 설정
save_columns = ['ID', 'Segment'] + important_features

# 컬럼 추출해 새 데이터프레임 생성
df_selected = df[save_columns].copy()
df_selected

,ID,Segment,OS구분코드,불만제기후경과월_R12M,방문월수_PC_R6M,인입일수_ARS_R6M,당사멤버쉽_방문월수_R6M,방문후경과월_앱_R6M,방문후경과월_PC_R6M,홈페이지_금융건수_R6M,이용메뉴건수_ARS_R6M_num,IB문의건수_CS_R6M,홈페이지_금융건수_R3M,인입일수_ARS_B0M,IB문의건수_사용승인내역_R6M,방문월수_앱_R6M,이용메뉴건수_ARS_B0M,인입월수_ARS_R6M,방문횟수_PC_B0M
0,TRAIN_000000,D,0,12,0,12,6,6,6,0,30,0,0,2,0,0,6,6,0
1,TRAIN_000001,E,2,12,0,0,0,6,6,0,1,0,0,0,0,0,0,0,0
2,TRAIN_000002,C,0,12,1,1,0,0,1,12,1,0,6,0,0,6,0,1,0
3,TRAIN_000003,D,0,12,0,12,6,6,6,0,30,0,0,2,0,0,6,6,0
4,TRAIN_000004,E,0,0,0,0,0,6,6,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,TEST_99995,None,2,0,0,0,0,6,6,0,1,0,0,0,0,0,0,0,0
499996,TEST_99996,None,2,12,0,0,0,0,6,3,1,0,3,0,0,6,0,0,0
499997,TEST_99997,None,2,0,0,0,0,6,6,0,1,0,0,0,0,0,0,0,0
499998,TEST_99998,None,0,12,0,0,0,0,6,7,1,0,4,0,0,6,0,0,0


In [10]:
# train과 test 모두 포함된 선택된 데이터 CSV 파일로 저장
df_selected.to_csv('채널정보_최종.csv', index=False, encoding='utf-8-sig')

### 보통 1%(=0.01) 이상이면 “모델에서 어느 정도 의미 있는 피처”로, 5%(=0.05) 이상이면 “꽤 중요한 피처